In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
import xarray as xr
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from FLplot import getFLpathData as getFL

warnings.filterwarnings("ignore",message='invalid value encountered in less_equal')

%matplotlib inline

In [2]:
savePath = '/Users/danstechman/GoogleDrive/School/Research/PECAN/Microphysics/plots/vertical_profiles'
fType = 'pdf'

chkLims     = False

noDispSave  = True

plotRHtemp  = True
plotTempAlt = True
plotRHAlt   = True
plotWSAlt   = True
plotWStemp  = True

# flights = ['20150617','20150620','20150701','20150702','20150706','20150709']
flights = ['20150706']

In [3]:
tempLim = (-18.5,22)
tdLim   = (-55,20)
rhLim   = (0,120)
altLim  = (1200,7500)
wsLim   = (0,45)

In [4]:
if chkLims:
    tmpTDmin = np.ones(42,)*np.nan
    tmpTDmax = np.ones(42,)*np.nan
    tmpAltMin = np.ones(42,)*np.nan
    tmpAltMax = np.ones(42,)*np.nan
    tmpWSmin = np.ones(42,)*np.nan
    tmpWSmax = np.ones(42,)*np.nan
    iSprl = 0

for flight in flights:
    print('Working on {}...'.format(flight))
    
    figSavePath = '{}/{}'.format(savePath,flight)
    if not os.path.exists(figSavePath):
        os.makedirs(figSavePath)
        
    cipFile = '/Users/danstechman/GoogleDrive/PECAN-Data/mp-data/' + flight + '/' + flight + '_CIPfit-spirals-10s1sAvg.nc'
    flFile = '/Users/danstechman/GoogleDrive/PECAN-Data/FlightLevelData/Processed/' + flight + '_FltLvl_Processed.nc'
    pecanPrmF = '/Users/danstechman/GoogleDrive/PECAN-Data/' + flight + '_PECANparams.nc'
    
    # Pull out FL data
    flData = xr.open_dataset(flFile,decode_times=False)
    timeSecs_FL = flData.time_secs_FL.data
    tempC = flData.TA.data
    tdC = flData.TD.data
    rh = flData.RH_hybrid.data
    alt_mMSL = flData.Alt.data
    windSpd_ms = flData.windSpd.data
    
    # Pull out any PECAN parameters
    pecanPrms = xr.open_dataset(pecanPrmF,decode_times=False)
    startT = pecanPrms.startT.data
    endT = pecanPrms.endT.data
    mlBotTemp = pecanPrms.mlBotTemp.data
    mlTopTemp = pecanPrms.mlTopTemp.data

    # Pull out any global variables/attributes from the netcdf file
    cipData_root = xr.open_dataset(cipFile)
    sprlZone = str(cipData_root.sprlZone.data,'utf-8')
    mcsType = str(cipData_root.mcsType.data,'utf-8')
    numSprls = len(sprlZone)

    # Loop over each spiral for the current flight
#     for ix in np.arange(0,numSprls):
    for ix in [4,5]:
        print('\tPlotting Spiral {}'.format(ix+1))
        
        # Get start and end indices for FL variables within current spiral
        strtMatch = min(timeSecs_FL, key=lambda x: abs(x - startT[ix]))
        endMatch = min(timeSecs_FL, key=lambda x: abs(x - endT[ix]))
        flStrtIx = np.squeeze(np.where(timeSecs_FL == strtMatch))
        flEndIx = np.squeeze(np.where(timeSecs_FL == endMatch))

######### Set plot-title-specific strings
        if mcsType[ix] == 'F':
            typStr = 'Trailing Strat. (formative)'
        if mcsType[ix] == 'M':
            typStr = 'Trailing Strat. (mature)'
        if mcsType[ix] == 'P':
            typStr = 'Parallel Strat.'
        if mcsType[ix] == 'L':
            typStr = 'Leading Strat.'
        if mcsType[ix] == 'C':
            typStr = 'Cluster MCS'
            
        if sprlZone[ix] == 'T':
            zoneStr = 'Transition Zone'
        if sprlZone[ix] == 'S':
            zoneStr = 'Enh. Strat. Rain Region'
        if sprlZone[ix] == 'A':
            zoneStr = 'Anvil Region'
        
        # RIJ spirals (all but #3 and #6 from 20 June)
        # (Remember that ix is 0-based, i.e., ix=2 is actually spiral 3)
        if flight == '20150620':
            if ix in [0,1]:
                rijStr = ' - RIJ Notch'
            elif ix in [3,4,6]:
                rijStr = ' - RIJ'
            else:
                rijStr = ''
                
            
            if ix in [0,1,2]:
                altElevAdj = 790
            elif ix in [3,4]:
                altElevAdj = 630
            elif ix == 5:
                altElevAdj = 565
            elif ix == 6:
                altElevAdj = 400
            else:
                altElevAdj = 0
                
            alt_mAGL = alt_mMSL - altElevAdj
        else:
            rijStr = ''
        
        if plotRHtemp:
            fig, ax = plt.subplots(figsize=(16,20))

            ax.axhline(y=mlTopTemp[ix],color='black',linestyle='--',linewidth=3,label='ML Top')
            ax.axhline(y=mlBotTemp[ix],color='red',linestyle='--',linewidth=3,label='ML Bottom')
            ax.axvline(x=100,color='#536872',linestyle='--',linewidth=3)
            ax.plot(rh[flStrtIx:flEndIx],tempC[flStrtIx:flEndIx],'-bo',linewidth=2,
                    markeredgecolor='white',markeredgewidth=0.5,markersize=6)

            
            ax.set_xlim(rhLim)
            ax.set_ylim(tempLim)
            ax.invert_yaxis()
            ax.set_xlabel('Relative Humidity (%)',fontsize=24)
            ax.set_ylabel('Temperature ($^{\circ}$C)',fontsize=24)
            ax.tick_params(axis='both', which='major', labelsize=22)
            ax.set_title('{} - RH - Spiral {}\n{} - {}{}'.format(flight,ix+1,typStr,zoneStr,rijStr),fontsize=26)
            ax.xaxis.set_minor_locator(MultipleLocator(10))
            ax.grid(which='both')
            ax.legend(loc='lower left',fontsize=18)
            # Save the output figure
            saveStr = '{}/{}_FL-RH-Temp_sprl{:02d}.{}'.format(figSavePath,flight,ix+1,fType)
            if noDispSave:
                fig.savefig(saveStr,bbox_inches='tight')
                plt.close(fig)
                
        if plotTempAlt:
            fig, ax = plt.subplots(figsize=(16,20))

            ax.axvline(x=mlTopTemp[ix],color='black',linestyle='--',linewidth=3,label='ML Top')
            ax.axvline(x=mlBotTemp[ix],color='red',linestyle='--',linewidth=3,label='ML Bottom')
            # ax.plot(tempC[flStrtIx:flEndIx],alt_mAGL[flStrtIx:flEndIx],color='#8b0000',linestyle='-',linewidth=2)
            ax.plot(tempC[flStrtIx:flEndIx],alt_mMSL[flStrtIx:flEndIx],color='#8b0000',linestyle='-',linewidth=2)#,
                    #marker='o',markeredgecolor='white',markeredgewidth=0.5,markersize=6)

            ax.set_xlim(tempLim)
            ax.set_ylim(altLim)
            #ax.set_ylabel('Altitude (m AGL)',fontsize=24)
            ax.set_ylabel('Altitude (m MSL)',fontsize=24)
            ax.set_xlabel('Temperature ($^{\circ}C$)',fontsize=24)
            ax.tick_params(axis='both', which='major', labelsize=22)
            ax.set_title('{} - Temperature - Spiral {}\n{} - {}{}'.format(flight,ix+1,typStr,zoneStr,rijStr),fontsize=26)
            ax.xaxis.set_minor_locator(MultipleLocator(10))
            ax.grid(which='both')
            ax.legend(loc='lower left',fontsize=18)
            # Save the output figure
            saveStr = '{}/{}_FL-Temp-Alt_sprl{:02d}.{}'.format(figSavePath,flight,ix+1,fType)
            if noDispSave:
                fig.savefig(saveStr,bbox_inches='tight')
                plt.close(fig)
                
        if plotRHAlt:
            fig, ax = plt.subplots(figsize=(16,20))


            #ax.plot(rh[flStrtIx:flEndIx],alt_mAGL[flStrtIx:flEndIx],color='b',linestyle='-',linewidth=2)
            ax.plot(rh[flStrtIx:flEndIx],alt_mMSL[flStrtIx:flEndIx],color='b',linestyle='-',linewidth=2)#,
                    #marker='o',markeredgecolor='white',markeredgewidth=0.5,markersize=6)

            ax.set_xlim(rhLim)
            ax.set_ylim(altLim)
            # ax.set_ylabel('Altitude (m AGL)',fontsize=24)
            ax.set_ylabel('Altitude (m MSL)',fontsize=24)
            ax.set_xlabel('Relative Humidity (%)',fontsize=24)
            ax.tick_params(axis='both', which='major', labelsize=22)
            ax.set_title('{} - RH - Spiral {}\n{} - {}{}'.format(flight,ix+1,typStr,zoneStr,rijStr),fontsize=26)
            ax.xaxis.set_minor_locator(MultipleLocator(10))
            ax.grid(which='both')
            # Save the output figure
            saveStr = '{}/{}_FL-RH-Alt_sprl{:02d}.{}'.format(figSavePath,flight,ix+1,fType)
            if noDispSave:
                fig.savefig(saveStr,bbox_inches='tight')
                plt.close(fig)
                
        if plotWSAlt:
            fig, ax = plt.subplots(figsize=(16,20))

            #ax.plot(windSpd_ms[flStrtIx:flEndIx],alt_mAGL[flStrtIx:flEndIx],color='#483d8b',linestyle='-',linewidth=2)
            ax.plot(windSpd_ms[flStrtIx:flEndIx],alt_mMSL[flStrtIx:flEndIx],color='#483d8b',linestyle='-',linewidth=2)#,
                    #marker='o',markeredgecolor='white',markeredgewidth=0.5,markersize=6)

            ax.set_xlim(wsLim)
            ax.set_ylim(altLim)
            #ax.set_ylabel('Altitude (m AGL)',fontsize=24)
            ax.set_ylabel('Altitude (m MSL)',fontsize=24)
            ax.set_xlabel('Wind Speed ($m\ s^{-1}$)',fontsize=24)
            ax.tick_params(axis='both', which='major', labelsize=22)
            ax.set_title('{} - Wind Speed - Spiral {}\n{} - {}{}'.format(flight,ix+1,typStr,zoneStr,rijStr),fontsize=26)
            ax.xaxis.set_minor_locator(MultipleLocator(10))
            ax.grid(which='both')
            # Save the output figure
            saveStr = '{}/{}_FL-WS-Alt_sprl{:02d}.{}'.format(figSavePath,flight,ix+1,fType)
            if noDispSave:
                fig.savefig(saveStr,bbox_inches='tight')
                plt.close(fig)
                
        if plotWStemp:
            fig, ax = plt.subplots(figsize=(16,20))

            ax.axhline(y=mlTopTemp[ix],color='black',linestyle='--',linewidth=3,label='ML Top')
            ax.axhline(y=mlBotTemp[ix],color='red',linestyle='--',linewidth=3,label='ML Bottom')
            ax.plot(windSpd_ms[flStrtIx:flEndIx],tempC[flStrtIx:flEndIx],color='#483d8b',linestyle='-',linewidth=2,
                    marker='o',markeredgecolor='white',markeredgewidth=0.5,markersize=6)

            
            ax.set_xlim(wsLim)
            ax.set_ylim(tempLim)
            ax.invert_yaxis()
            ax.set_xlabel('Wind Speed ($m\ s^{-1}$)',fontsize=24)
            ax.set_ylabel('Temperature ($^{\circ}$C)',fontsize=24)
            ax.tick_params(axis='both', which='major', labelsize=22)
            ax.set_title('{} - Wind Speed - Spiral {}\n{} - {}{}'.format(flight,ix+1,typStr,zoneStr,rijStr),fontsize=26)
            ax.xaxis.set_minor_locator(MultipleLocator(10))
            ax.grid(which='both')
            ax.legend(loc='lower left',fontsize=18)
            # Save the output figure
            saveStr = '{}/{}_FL-WS-Temp_sprl{:02d}.{}'.format(figSavePath,flight,ix+1,fType)
            if noDispSave:
                fig.savefig(saveStr,bbox_inches='tight')
                plt.close(fig)
            
        if chkLims:
            tmpTDmin[iSprl] = np.nanmin(tdC[flStrtIx:flEndIx])
            tmpTDmax[iSprl] = np.nanmax(tdC[flStrtIx:flEndIx])
            tmpAltMin[iSprl] = np.nanmin(alt_mMSL[flStrtIx:flEndIx])
            tmpAltMax[iSprl] = np.nanmax(alt_mMSL[flStrtIx:flEndIx])
            tmpWSmin[iSprl] = np.nanmin(windSpd_ms[flStrtIx:flEndIx])
            tmpWSmax[iSprl] = np.nanmax(windSpd_ms[flStrtIx:flEndIx])
            iSprl += 1
if chkLims:
    print('Min dewpoint temp: {}C'.format(np.min(tmpTDmin)))
    print('Max dewpoint temp: {}C'.format(np.max(tmpTDmax)))
    print('Min altitude: {} m MSL'.format(np.min(tmpAltMin)))
    print('Max altitude: {} m MSL'.format(np.max(tmpAltMax)))
    print('Min windspeed: {} m/s'.format(np.min(tmpWSmin)))
    print('Max windspeed: {} m/s'.format(np.max(tmpWSmax)))

Working on 20150706...
	Plotting Spiral 5
	Plotting Spiral 6
